In [1]:
import pandas as pd
from matplotlib import pyplot as plt

from gensim.models.word2vec import Word2Vec
from os import path, mkdir
from multiprocessing import cpu_count

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize, regexp_tokenize
from konlpy.tag import Kkma,Twitter
from collections import Counter

import re

C:\Users\Chankoo\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
d = pd.read_excel('test.xlsx',sheet_name='sheet1')

In [3]:
d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8035 entries, 0 to 8034
Data columns (total 13 columns):
연월      8035 non-null int64
전시제목    8035 non-null object
링크      8035 non-null object
전시작가    8035 non-null object
전시일정    8035 non-null object
초대일시    7629 non-null object
관람시간    8034 non-null object
전시장소    8034 non-null object
주소      8034 non-null object
연락처     8034 non-null object
홈페이지    7432 non-null object
아티클     8034 non-null object
텍스트     7703 non-null object
dtypes: int64(1), object(12)
memory usage: 878.8+ KB


In [4]:
d.dropna(subset=['텍스트'],inplace=True) #텍스트 없는 로우 드랍

In [5]:
sum(d.duplicated(subset=['텍스트'])) # 중복된 텍스트

20

In [6]:
toDropIndex = d[d.duplicated(subset=['텍스트'])].index

In [7]:
d.drop(index = toDropIndex, inplace=True)

In [8]:
d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7683 entries, 0 to 8034
Data columns (total 13 columns):
연월      7683 non-null int64
전시제목    7683 non-null object
링크      7683 non-null object
전시작가    7683 non-null object
전시일정    7683 non-null object
초대일시    7292 non-null object
관람시간    7683 non-null object
전시장소    7683 non-null object
주소      7683 non-null object
연락처     7683 non-null object
홈페이지    7103 non-null object
아티클     7683 non-null object
텍스트     7683 non-null object
dtypes: int64(1), object(12)
memory usage: 840.3+ KB


In [ ]:
# kkma = Kkma()
# tokens = kkma.morphs(corpus) 

twitter = Twitter()
# tokens = twitter.pos(corpus)# 찢어서하자

In [12]:
# corpus = ''
# for txt in d.텍스트.iloc[:100]:
#     if txt==None:
#         print('None')
#         continue
#     corpus +=str(txt)

In [31]:
# tokens = []
# for txt in d.텍스트.iloc[:100]:
#     if txt==None:
#         continue
#     pos = ['/'.join(pos) for pos in twitter.pos(txt)]

#     tokens.append(pos)
    

In [56]:
tokens = []
for txt in d.텍스트:
    if txt==None:
        continue
    tokens.append(twitter.morphs(txt)) #morphs 만 가져오자
    

In [59]:
len(tokens) #doc 갯수

7683

In [72]:
len(tokens[0]) # 한 doc의 token 갯수

349

In [47]:
#하이퍼파라미터 세팅
num_features = 100
min_word_count = 10
num_workers = cpu_count()
context_size = 10
downsampling = 1e-3
seed=180816

In [60]:
#모델 생성
art2vec =Word2Vec( 
    sg=1, #skip-gram
    workers=cpu_count(),
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample = downsampling,
)

In [61]:
art2vec.build_vocab(tokens)

In [62]:
art2vec.corpus_count

7683

In [63]:
art2vec.epochs

5

In [64]:
art2vec.train(tokens,total_examples=art2vec.corpus_count,epochs=art2vec.epochs) #학습

(20612852, 28226735)

In [65]:
##모델을 저장하고 불러와서 다시 training시킬 수 있다
art2vec.save(r'''art2vec.w2v''')

In [ ]:
##전에 학습한 결과 불러오기
art2vec.load(r'''art2vec.w2v''')

In [67]:
art2vec.wv.most_similar('광주')

[('전남', 0.7835304737091064),
 ('시립', 0.7778898477554321),
 ('예향', 0.7698221206665039),
 ('창원', 0.7405427694320679),
 ('송영옥', 0.7315448522567749),
 ('성곡', 0.7311973571777344),
 ('전북', 0.7283378839492798),
 ('은암', 0.7268154621124268),
 ('전당', 0.7249138951301575),
 ('황영', 0.7231484651565552)]

In [68]:
art2vec.wv.most_similar('희망')

[('희망이', 0.7420464754104614),
 ('약속하는', 0.7306351661682129),
 ('소망', 0.7047438621520996),
 ('희구', 0.7004313468933105),
 ('행복', 0.6934530735015869),
 ('건강하고', 0.6868695020675659),
 ('행복함', 0.6806673407554626),
 ('긍정', 0.672339677810669),
 ('갈망', 0.6695137619972229),
 ('절망', 0.6681349277496338)]

In [69]:
art2vec.wv.most_similar('가족')

[('이웃', 0.7234182357788086),
 ('반려동물', 0.7176386117935181),
 ('가정', 0.69026118516922),
 ('부모', 0.6825472116470337),
 ('연인', 0.6782400608062744),
 ('부모님', 0.6771761178970337),
 ('자식', 0.6764198541641235),
 ('반려', 0.6757124662399292),
 ('친구', 0.6733570098876953),
 ('단란', 0.6727656722068787)]

In [70]:
art2vec.wv.most_similar('재미')

[('즐거움', 0.7512965202331543),
 ('유쾌', 0.6559067964553833),
 ('친근함', 0.6535444259643555),
 ('최순민', 0.641376256942749),
 ('재미있고', 0.6413453221321106),
 ('재미있는', 0.6399206519126892),
 ('청량감', 0.6387925148010254),
 ('재미있게', 0.6228461265563965),
 ('색다른', 0.6178460121154785),
 ('선물하는', 0.6155192852020264)]

In [73]:
art2vec.wv.most_similar('최순민')

[('전달해준', 0.7862638235092163),
 ('엉뚱하고', 0.7628570795059204),
 ('소소함', 0.7452802658081055),
 ('사육사', 0.7437958717346191),
 ('볼라뇨', 0.7396140098571777),
 ('유발하고', 0.7381994724273682),
 ('좋아할', 0.7375392317771912),
 ('가져다주는', 0.7372032999992371),
 ('화사함', 0.7319670915603638),
 ('김섭', 0.730377197265625)]

In [75]:
art2vec.wv.most_similar('박경태')

[('실현된', 0.6371790766716003),
 ('조이경', 0.636550784111023),
 ('불가능', 0.619219183921814),
 ('현현', 0.618384599685669),
 ('불완전하고', 0.617784857749939),
 ('도래', 0.6132364869117737),
 ('정성윤', 0.6103134155273438),
 ('휘발', 0.6101908683776855),
 ('아포리아', 0.6101325154304504),
 ('거스', 0.610127866268158)]

In [76]:
art2vec.wv.most_similar('정성윤')

[('조이경', 0.7433583736419678),
 ('김수연', 0.7237849235534668),
 ('신성환', 0.719285786151886),
 ('보비', 0.7103101015090942),
 ('정석희', 0.7096210718154907),
 ('이문호', 0.7096121311187744),
 ('빠키', 0.7055789232254028),
 ('심포니', 0.6999529600143433),
 ('요나', 0.6984244585037231),
 ('매핑', 0.6980416774749756)]